In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import sys
sys.executable

'/home/peromoseq/anaconda3/envs/mmdeploy/bin/python3'

### important
- must be `module load cuda/11.7`

##### Duration: ???

In [4]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


In [5]:
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Python interpreter binary location: /home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


### Get recording info (google sheets)

In [6]:
import requests
import pandas as pd
from io import BytesIO

In [7]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [8]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Run 2D prediction

In [9]:
for idx, recording_row in recording_df.iterrows():
    break

In [10]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [11]:
from multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt import RTMModelConverter
output_directory_tensorrt = output_directory / 'tensorrt'
output_directory_tensorrt.mkdir(parents=True, exist_ok=True)

# https://o2portal.rc.hms.harvard.edu/node/compute-g-17-148.o2.rc.hms.harvard.edu/59141/notebooks/tim_sainburg/projects/23-09-29-peromoseq/notebooks/keypoints/mmpose/24-01-11-deploy-2d-predictions-on-local/24-05-08-convert-pose-model.ipynb
rtmpose_model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524'
path_to_rmpose_config = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py')
path_to_rmpose_checkpoint = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth')

# https://o2portal.rc.hms.harvard.edu/node/compute-g-17-148.o2.rc.hms.harvard.edu/59141/notebooks/tim_sainburg/projects/23-09-29-peromoseq/notebooks/keypoints/mmpose/24-01-11-deploy-2d-predictions-on-local/240508-convert-detection-model-fp32.ipynb
rtmdetection_model_name = 'rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661'
path_to_rtmdetection_config = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py')
path_to_rtmdetection_checkpoint = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth')

model_converter = RTMModelConverter(
    path_to_rmpose_config = path_to_rmpose_config,
    path_to_rmpose_checkpoint = path_to_rmpose_checkpoint,
    path_to_rtmdetection_config = path_to_rtmdetection_config,
    path_to_rtmdetection_checkpoint = path_to_rtmdetection_checkpoint,
    tensorrt_output_directory = output_directory_tensorrt,
    rtmdetection_model_name = rtmdetection_model_name,
    rtmpose_model_name = rtmpose_model_name,
    conda_env = "/home/peromoseq/anaconda3/envs/mmdeploy",
    is_local=True
)

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Python interpreter binary location: /home/peromoseq/anaconda3/envs/mmdeploy/bin/python3
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Using CUDA device: NVIDIA_GeForce_RTX_4070_Ti


In [13]:
output_directory_tensorrt

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Python interpreter binary location: /home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt')

In [12]:
model_converter.run()

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:TensorRT detection model already exists.


source $(conda info --base)/etc/profile.d/conda.sh;
conda activate /home/peromoseq/anaconda3/envs/mmdeploy;
export PYTHONPATH=/tmp/tmpz7gn981u:$PYTHONPATH;
python /n/groups/datta/tim_sainburg/projects/mmdeploy/tools/deploy.py /n/groups/datta/tim_sainburg/projects/mmdeploy/configs/mmpose/pose-detection_simcc_tensorrt_dynamic-256x256.py /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/example_data/test_mouse_cropped.png --work-dir /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/NVIDIA_GeForce_RTX_4070_Ti --device cuda:0 --show --dump-info
08/13 14:15:03 

08/13 14:17:51 - mmengine - INFO - visualize tensorrt model success.
08/13 14:17:51 - mmengine - INFO - visualize pytorch model start.
08/13 14:17:57 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
08/13 14:17:57 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "mmpose_tasks" registry tree. As a workaround, the current "mmpose_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.


KeyboardInterrupt: 

In [ ]:
output_directory_2D = output_directory/ "2D_predictions" 

In [ ]:
output_directory_2D.mkdir(parents=True, exist_ok=True)

In [ ]:
recording_row.video_location_on_o2

In [ ]:
video_directory = Path(recording_row.video_location_on_o2) / recording_row.video_recording_id

In [ ]:
pose_checkpoint = "/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth"
pose_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py'
det_checkpoint = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth'
det_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py'

In [ ]:
#tensorrt_path = "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt"
rtmpose_model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524'
rtmdetection_model_name = 'rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661'

In [ ]:
output_directory_predictions = output_directory_2D / recording_row.video_recording_id

In [ ]:
tensorrt_model_directory = output_directory / "tensorrt"

In [ ]:
output_directory_predictions = output_directory_2D / (recording_row.video_recording_id + '_tensorrt')

In [ ]:
output_directory_predictions

In [ ]:
expected_video_length_frames = recording_row.max_video_duration_m * recording_row.samplerate

In [ ]:
inferencer = Inferencer2D(
    video_directory,
    output_directory_predictions = output_directory_predictions,
    pose_estimator_config = pose_config,
    pose_estimator_checkpoint = pose_checkpoint,
    detector_config = det_config,
    detector_checkpoint = det_checkpoint,
    tensorrt_rtmdetection_model_name = rtmdetection_model_name,
    tensorrt_rtmpose_model_name = rtmpose_model_name,
    tensorrt_model_directory = tensorrt_model_directory,
    use_tensorrt = True,
)

In [ ]:
inferencer.run()